In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-10-22"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
8414,2024-10-22,Brasil Nbb,19:30,Pato,Fortaleza B.C.,1.59,2.24,154.5,1.85,1.85,-4.5,1.96,1.75,EexLarKN,0.628931,0.446429,0.540541,0.540541,0.075359,0.6,0.6,NaN,NaN,160.460,38.527402,0.240106,119.810,31.185772,0.260294,142.750,180.006,166.05,141.96,0.0,0.0,0.0,0.0,0.240010,0.000000,0.080050,-3.64,-0.728,-0.810440,0.522801,0.4,-0.122801,1.33,0.266,4.661654,0.398510,0.3,-0.098510
8415,2024-10-22,Brasil Nbb,20:00,Botafogo,Corinthians Paulista,2.07,1.69,154.5,1.85,1.85,1.5,1.96,1.75,QZ5wIPZi,0.483092,0.591716,0.540541,0.540541,0.074808,0.4,0.6,NaN,NaN,294.732,159.960105,0.542731,163.646,51.735931,0.316145,309.634,174.266,557.67,177.30,1.0,0.0,0.0,0.0,0.142926,0.000000,0.080050,-2.82,-0.564,-1.897163,0.408568,0.4,-0.008568,2.72,0.544,1.268382,0.470912,0.5,0.029088
8416,2024-10-22,Brasil Nbb,20:00,São Paulo,Franca,2.66,1.43,160.5,1.82,1.88,4.5,1.96,1.75,pvTjfMso,0.375940,0.699301,0.549451,0.531915,0.075241,0.8,0.4,NaN,NaN,179.628,99.340027,0.553032,127.366,30.496924,0.239443,149.698,135.682,115.62,156.62,0.0,0.0,0.0,0.0,0.425301,0.022933,0.080050,-2.91,-0.582,-2.852234,0.609634,0.3,-0.309634,1.61,0.322,1.335404,0.647835,0.8,0.152165
8417,2024-10-22,Eua Nba,20:30,Boston Celtics,New York Knicks,1.46,2.86,221.5,1.88,1.98,-5.5,1.93,1.93,0ptyAqOi,0.684932,0.349650,0.531915,0.505051,0.034582,1.0,0.6,NaN,NaN,179.050,33.127519,0.185018,163.776,28.932563,0.176659,196.240,232.196,159.85,133.38,0.0,0.0,0.0,0.0,0.458310,0.036638,0.000000,-1.04,-0.208,-2.211538,0.760182,0.8,0.039818,-2.82,-0.564,-3.297872,0.490455,0.5,0.009545
8418,2024-10-22,Eua Nba,23:00,Los Angeles Lakers,Minnesota Timberwolves,1.93,1.93,218.5,1.84,2.00,1.5,1.83,2.04,MaUq8Nh4,0.518135,0.518135,0.543478,0.500000,0.036269,0.6,0.6,NaN,NaN,262.676,78.513910,0.298900,221.118,40.563997,0.183450,220.496,231.372,220.80,188.19,0.0,0.0,0.0,0.0,0.000000,0.058926,0.076740,-1.05,-0.210,-4.428571,0.541683,0.6,0.058317,-0.01,-0.002,-465.000000,0.493680,0.5,0.006320
8419,2024-10-22,Europa Eurocopa,13:00,Lietkabelis,JL Bourg,3.77,1.24,156.5,1.85,1.85,8.5,1.91,1.80,vNRgCxt1,0.265252,0.806452,0.540541,0.540541,0.071704,1.0,0.2,NaN,NaN,252.308,105.793484,0.419303,136.060,44.204119,0.324887,187.450,125.976,116.92,147.84,0.0,0.0,0.0,0.0,0.714164,0.000000,0.041931,-0.78,-0.156,-17.756410,0.525419,0.5,-0.025419,1.31,0.262,0.916031,0.589907,0.7,0.110093
8420,2024-10-22,Europa Eurocopa,14:00,Besiktas,Joventut,1.36,2.97,162.5,1.85,1.85,-7.5,2.01,1.72,S8iu7s4L,0.735294,0.336700,0.540541,0.540541,0.071994,0.8,0.4,NaN,NaN,147.368,40.591882,0.275446,129.592,46.722771,0.360537,156.872,173.086,149.80,103.74,0.0,0.0,0.0,0.0,0.525839,0.000000,0.109952,-2.17,-0.434,-0.829493,0.666589,0.5,-0.166589,-1.99,-0.398,-4.949749,0.554177,0.6,0.045823
8421,2024-10-22,Europa Eurocopa,15:00,Venezia,Hamburg,1.13,5.45,166.5,1.85,1.85,-12.5,1.92,1.78,OlQMJKmR,0.884956,0.183486,0.540541,0.540541,0.068442,0.6,0.8,NaN,NaN,271.560,316.918149,1.167028,286.806,183.669884,0.640398,147.648,587.640,138.60,273.06,0.0,0.0,0.0,0.0,0.928481,0.000000,0.053511,0.04,0.008,16.250000,0.000000,0.0,0.000000,-5.00,-1.000,-4.450000,0.174306,0.2,0.025694
8422,2024-10-22,Europa Eurocopa,16:00,Gran Canaria,Ulm,1.13,5.40,166.5,1.83,1.83,-12.5,1.92,1.78,ngYB1LRr,0.884956,0.185185,0.546448,0.546448,0.070141,0.6,0.6,NaN,NaN,130.638,48.865279,0.374051,187.386,33.767805,0.180205,98.712,192.760,97.02,207.20,0.0,0.0,0.0,0.0,0.924761,0.000000,0.053511,-2.39,-0.478,-0.271967,0.866957,0.7,-

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,20:30,Eua Nba,Boston Celtics,New York Knicks,221.5,1.88,0.9955,Over
1,23:00,Eua Nba,Los Angeles Lakers,Minnesota Timberwolves,218.5,1.84,0.9558,Over
2,15:45,Espanha Copa España,Basquet L'Horta Godella,Alicante,152.5,1.83,0.9991,Over
